<a href="https://colab.research.google.com/github/vchiang001/sexual_diversity_dissertation/blob/main/simba_smoothing.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import os

In [ ]:
mi_succ = 100 #set rolling window (mount_intromit)
mi_ratio = 0.3 #set as ratio of frames indicating presence of behaviour (mount_intromit)
ej_succ = 100 #set rolling window (ejaculate)
ej_ratio = 0.3 #set as ratio of frames indicating presence of behaviour (ejaculate)
machine_directory = '/content/drive/MyDrive/BEHAVIOMICS/SpecificAim1/lowres_simba'

In [ ]:
#function to smooth the data
def transform_data(data, succ, ratio):
    transformed_data = []
    prev_value = None
    for i in range(len(data)):
        if data[i] == 1:
            if i == 0:
                if sum(data[i:i + succ]) < ratio * succ:
                    transformed_data.append(0)
                else:
                    transformed_data.append(1)
            elif prev_value != 1:
                transformed_data.append(0)
            elif sum(data[i:i + succ]) < ratio * succ:
                transformed_data.append(0)
            else:
                transformed_data.append(1)
        elif data[i] == 0:
            if i != 0:
                if sum(data[i:i + succ]) > ratio * succ:
                    transformed_data.append(1)
                else:
                    transformed_data.append(0)
            elif prev_value == 1:
                transformed_data.append(1)
            elif sum(data[i:i + succ]) > ratio * succ:
                transformed_data.append(1)
            else:
                transformed_data.append(0)
        prev_value = transformed_data[-1]
    return transformed_data

In [ ]:
# Iterate over each file in the directory
for file_name in os.listdir(machine_directory):
    # Check if the file is a CSV file
    if file_name.endswith(".csv"):
        print(file_name)
        df = pd.read_csv(os.path.join(machine_directory, file_name))

        #Smoothing MI
        extracted_data = df.loc[:8516, "mount_intromit"]
        print(extracted_data)
        transformed_data = transform_data(data=extracted_data, succ=mi_succ, ratio=mi_ratio)
        transformed_df = pd.DataFrame(transformed_data, columns=['transformed_data'])
        print(transformed_df)
        transformed_data_values = transformed_df['transformed_data'].values
        transformed_data_values_adjusted = transformed_data_values[:8516]
        df.loc[:8515, "mount_intromit"] = transformed_data_values_adjusted

        #Smoothing EJ
        extracted_data = df.loc[:8516, "ejaculate"]
        print(extracted_data)
        transformed_data = transform_data(data=extracted_data, succ=ej_succ, ratio=ej_ratio)
        transformed_df = pd.DataFrame(transformed_data, columns=['transformed_data'])
        print(transformed_df)
        transformed_data_values = transformed_df['transformed_data'].values
        transformed_data_values_adjusted = transformed_data_values[:8516]
        df.loc[:8515, "ejaculate"] = transformed_data_values_adjusted

        # Save the DataFrame to a CSV file with the corresponding file name
        df.to_csv("/content/drive/MyDrive/BEHAVIOMICS/SpecificAim1/lowres_simba_transformed/" + file_name, index=False)